Este cuaderno es una demo del modulo _Simulador_ y el modulo de optizacion (_Workforce_), que usa este para computar una estrategia optima. 

# Simulador
## para simulación: Clase ``MisEscritorios_v03`` y funcion ``optuna_simular_v03``
## para optuna: Clase ``MisEscritorios_v02`` y funcion ``optuna_simular_v02``

- `MisEscritorios` contiene todo lo respectivo a planificación. 
- `optuna_simular_v02` es la función que corre una simulación, generando una tabla de atenciones. 


In [1]:
import os
os.chdir("/DeepenData/Repos/Flux_v0")
from typing import Dict, List, Tuple
import random
from copy import deepcopy
from itertools import count, islice
import numpy as np
from dev.atributos_de_series import atributos_x_serie
import copy
from src.datos_utils import DatasetTTP, obtener_skills

from src.optuna_utils import (
    sla_x_serie, 
    calculate_geometric_mean, 
    extract_skills_length, 
    extract_min_value_keys, 
    extract_max_value_keys, 
    non_empty_subsets, 
    get_random_non_empty_subset, 
    get_time_intervals,
    partition_dataframe_by_time_intervals,  
    plan_unico
    )

from src.simulador_v02 import (
    generate_integer, 
    actualizar_conexiones,
    generador_emisiones,
    timestamp_iterator,
    terminar_un_tramo,
    iniciar_un_tramo,
    update_escritorio,
    separar_por_conexion,
    poner_pasos_alternancia,
    pasos_alternancia,
    mismo_minuto,
    balancear_carga_escritorios,
    extract_highest_priority_and_earliest_time_row,
    remove_selected_row,
    FIFO
    ) 
import pandas as pd

from dev.pasos_alternancia_y_prioridades_x_escri import (
    generar_pasos_para_alternancia_v02,
    pasos_alternancia_v02,
    poner_pasos_alternancia_v02,
    MisEscritorios_v03,
    optuna_simular_v03
    )

/DeepenData/.miniconda/envs/flux/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Simulación con Clase ``MisEscritorios_v03`` y funcion ``optuna_simular_v03``

In [3]:
dataset = DatasetTTP.desde_csv_atenciones("data/fonasa_monjitas.csv.gz")
un_dia = dataset.un_dia("2023-05-15").sort_values(by='FH_Emi', inplace=False)
skills   = obtener_skills(un_dia)
series   = sorted(list({val for sublist in skills.values() for val in sublist}))
modos    = ['Rebalse','Alternancia', 'Rebalse']
atributos_series = atributos_x_serie(ids_series=series, 
                                    sla_porcen_user=None, 
                                    sla_corte_user=None, 
                                    pasos_user=None, 
                                    prioridades_user=None)
niveles_servicio_x_serie = {atr_dict['serie']:
                            (atr_dict['sla_porcen']/100, atr_dict['sla_corte']/60) 
                            for atr_dict in atributos_series}

planificacion = {'0': [{'inicio': '08:40:11',
    'termino': '10:07:40',
    'propiedades': {'skills' : get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
            'atributos_series':atributos_series,
            
        }}],
    '1': [{'inicio': '08:40:11',
    'termino': '10:07:40',
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '12': [{'inicio': '08:40:11',
    'termino': '10:07:40',
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '33': [{'inicio': '11:36:03',
    'termino': '13:02:33',
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '34': [{'inicio': '11:36:03',
    'termino': '13:02:33',
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '35': [{'inicio': '11:36:03',
    'termino': '13:02:33',
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '49': [{'inicio': '13:02:56',
    'termino': '14:30:23',
    'propiedades': {'skills': get_random_non_empty_subset(series), 
        'configuracion_atencion':random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '50': [{'inicio': '13:02:56',
    'termino': '14:30:23',
    'propiedades': {'skills': get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
            'atributos_series':atributos_series,

        }}],
    '51': [{'inicio': '13:02:56',
    'termino': '14:30:23',
    'propiedades': {'skills':get_random_non_empty_subset(series),
        'configuracion_atencion': random.sample(modos, 1)[0],
        'porcentaje_actividad'  : np.random.randint(75, 90)/100,
        'atributos_series':atributos_series,
        }}]}


optuna_simular_v03(planificacion, niveles_servicio_x_serie, un_dia, tipo_inactividad = 'Porcentaje' )

(                  FH_Emi IdSerie espera IdEsc T_Ate
 0    2023-05-15 08:40:11      12      0     7    96
 1    2023-05-15 08:40:54      14      0    10   157
 2    2023-05-15 08:41:07      14      0    12   458
 3    2023-05-15 08:41:18      14      0    11   861
 4    2023-05-15 08:41:27      14      0    10   381
 ..                   ...     ...    ...   ...   ...
 654  2023-05-15 14:12:59      10     18   NaN   NaN
 655  2023-05-15 14:13:23      10     17   NaN   NaN
 656  2023-05-15 14:14:26      12     16   NaN   NaN
 657  2023-05-15 14:18:32      10     12   NaN   NaN
 658  2023-05-15 14:30:23      17      0   NaN   NaN
 
 [659 rows x 5 columns],
 498)

## OPTUNA con Clase ``MisEscritorios_v02`` y funcion ``optuna_simular_v02``

In [4]:

  
class MisEscritorios_v02:
    
    def __init__(self,
                 inicio_tramo:  pd.Timestamp, 
                 fin_tramo:     pd.Timestamp,
                 planificacion: dict, 
                 conexiones:    dict = None,
                 niveles_servicio_x_serie=None,
                 ):
      
        self.niveles_servicio_x_serie = niveles_servicio_x_serie
        self.planificacion = planificacion

        self.escritorios = {k: {  # Dictionary comprehension starts; k is the key, and the value is another nested dictionary.
                                    'estado': 'disponible',  # Assigns the string 'disponible' to the key 'estado'.
                                    'tiempo_actual_disponible': 0,  # Initializes 'tiempo_actual_disponible' to 0.
                                    'skills': v[0]['propiedades'].get('skills'),  # Uses .get() to safely extract 'skills' from 'propiedades'.
                                    
                                    'configuracion_atencion': v[0]['propiedades'].get('configuracion_atencion'),  # Similar to 'skills', safely extracts 'configuracion_atencion'.
                                    'contador_tiempo_disponible': iter(count(start=0, step=1)),  # Creates an iterator using Python's itertools.count, starting from 0 and incrementing by 1.

                                    'numero_de_atenciones': 0,  # Initializes 'numero_de_atenciones' to 0.
                                    
                                    # Tries to safely extract 'porcentaje_actividad' from 'propiedades' using .get().
                                    'porcentaje_actividad': v[0]['propiedades'].get('porcentaje_actividad'),

                                    # Checks if 'porcentaje_actividad' exists, and if not, sets 'duracion_inactividad' to None.
                                    'duracion_inactividad': int(
                                        (1 - v[0]['propiedades'].get('porcentaje_actividad', 0)) * (fin_tramo - inicio_tramo).total_seconds() / 60
                                    ) if v[0]['propiedades'].get('porcentaje_actividad') is not None else None,
                                    
                                    # Checks if 'porcentaje_actividad' exists, and if not, sets 'contador_inactividad' to None.
                                    'contador_inactividad': iter(islice(
                                        count(start=0, step=1),
                                        int((1 - v[0]['propiedades'].get('porcentaje_actividad', 0)) * (fin_tramo - inicio_tramo).total_seconds() / 60)
                                    )) if v[0]['propiedades'].get('porcentaje_actividad') is not None else None,
                                    
                                    'duracion_pausas': (1, 4, 47),  # Tuple containing min, avg, and max pause durations based on historical data.
                                    'probabilidad_pausas': .5,  # Probability that a pause will occur, again based on historical data.
                                    'numero_pausas': None  # Initializes 'numero_pausas' to None.
                                }
                                for k, v in self.planificacion.items()}  # The loop iterates over each key-value pair in self.planificacion.

        if not conexiones:
        #     #si no se provee el estado de los conexiones se asumen todas como True (todos conectados):
             conexiones                         = {f"{key}": random.choices([True, False], [1, 0])[0] for key in self.escritorios}
        self.escritorios                        = actualizar_conexiones(self.escritorios, conexiones)       
        self.escritorios_OFF                    = self.escritorios
        self.escritorios_ON                     = {}
        self.nuevos_escritorios_programados     = []
        self.registros_escritorios              = []
    def iniciar_atencion(self, escritorio, cliente_seleccionado):
        # iterar los escritorios y emisiones
        #for escr_bloq, emi in zip(escritorios_a_bloqueo, emision):
            
            #extraer los minutos que dura la atención asociada a la emision
        minutos_atencion = round((cliente_seleccionado.FH_AteFin - cliente_seleccionado.FH_AteIni).total_seconds()/60)
            #reescribir campos:
            
        self.escritorios_ON[escritorio]['contador_tiempo_atencion'] = iter(islice(count(start=0, step=1), minutos_atencion))#nuevo contador de minutos limitado por n_minutos
        self.escritorios_ON[escritorio]['estado']           = 'atención'#estado bloqueado significa que está atendiendo al cliente.
        self.escritorios_ON[escritorio]['minutos_atencion']  = minutos_atencion#tiempo de atención     
        self.escritorios[escritorio]['numero_de_atenciones'] += 1 #se guarda en self.escritorios para que no se resetee.
        self.escritorios_ON[escritorio]['numero_de_atenciones'] = self.escritorios[escritorio]['numero_de_atenciones'] 
    def filtrar_x_estado(self, state: str):   
        """
        extrae los escritorios por el estado (disponible o bloqueado)
        """     
        #obtener estados
        self.estados = {escr_i: {'estado': propiedades['estado'], 'configuracion_atencion': 
                        propiedades['configuracion_atencion']} for escr_i, propiedades in self.escritorios_ON.items()} 
        #extraer por disponibilidad    
        if disponibilidad := [
            key for key, value in self.estados.items() if value['estado'] == state
        ]:
            return disponibilidad
        else:
            #print(f"No hay escritorio {state}")
            return False
    def cambiar_propiedades_escritorio(self, 
                                       escritorio:str, 
                                       skills:List[int]=None, 
                                       configuracion_atencion:str=None, 
                                       conexion:bool=None, 
                                       duracion_pausas: tuple = None, # min_val:int=None, avg_val:int=None, max_val:int=None, 
                                       probabilidad_pausas:float=None,
                                       porcentaje_actividad=None) -> None:
        """_summary_
        Modifica las propiedades del escritorio. Si una propiedad entra vacía se ignora. 
        Args:
            escritorio (str): key del escritorio a modificar.
            skills (List[int], optional): Nueva lista de series para cargar como skills. Defaults to None.
            configuracion_atencion (str, optional): Nueva configuracion de atención. Defaults to None.
            conexion (bool, optional): Nuevo estado de conexion. Defaults to None.
        """

        campos = {
                  'skills': skills,
                  'configuracion_atencion': configuracion_atencion,
                  'conexion': conexion,
                  'duracion_pausas': duracion_pausas, #(min_val, avg_val, max_val),
                  'probabilidad_pausas': probabilidad_pausas,
                  'porcentaje_actividad':porcentaje_actividad,
                  }
        #remover propiedades del escritorio que no se modifican
        campos = {k: v for k, v in campos.items() if v is not None}
        #actualizar escritorio
        update_escritorio(escritorio, campos, self.escritorios_OFF, self.escritorios_ON)
        #print(f"{campos} de {escritorio} fue modificado.")
    def actualizar_conexiones_y_propiedades(self, un_escritorio, tramo, accion):
        
        
        propiedades = tramo['propiedades'] | {'conexion': accion == 'iniciar'}
    
        self.cambiar_propiedades_escritorio(un_escritorio, **propiedades)
        
        self.escritorios_ON, self.escritorios_OFF = separar_por_conexion(
            {**self.escritorios_ON, **self.escritorios_OFF}
        )
        
        self.escritorios_ON  = poner_pasos_alternancia(self.escritorios_ON, pasos_alternancia, self.niveles_servicio_x_serie)
    def aplicar_agenda(self, hora_actual, agenda):
        
        for idEsc, tramos_un_escritorio in agenda.items():
            
            if tramo_idx_tramo := terminar_un_tramo(hora_actual, tramos_un_escritorio):
                tramo     = tramo_idx_tramo[0]
                idx_tramo = tramo_idx_tramo[1]
                #print(f"{idEsc} termina tramo (eliminado de agenda): {tramo}")
                self.actualizar_conexiones_y_propiedades(idEsc, tramo, 'terminar')
                del agenda[idEsc][idx_tramo]   
            
            if tramo:=  iniciar_un_tramo(hora_actual, tramos_un_escritorio):
                #se va seguir ejecutando mientras el tramo sea válido
                #poner alguna flag para q no se vuelva a ejecutar
                #print(f"{idEsc} inicia tramo: {tramo}")
                self.actualizar_conexiones_y_propiedades(idEsc, tramo, 'iniciar')
    def iniciar_pausa(self, escritorio, tipo_inactividad:str = "Porcentaje", generador_pausa = generate_integer):
        # sourcery skip: extract-method, move-assign
      
        if tipo_inactividad == "Porcentaje":            
            self.escritorios_ON[escritorio]['estado'] = 'pausa'            
        else:
            min_val, avg_val, max_val = self.escritorios_ON[escritorio]['duracion_pausas']
            probabilidad_pausas       = self.escritorios_ON[escritorio]['probabilidad_pausas']
            minutos_pausa             = generador_pausa(min_val, avg_val, max_val, probabilidad_pausas)

            self.escritorios_ON[escritorio]['contador_tiempo_pausa'] = iter(islice(count(start=0, step=1), minutos_pausa))#nuevo contador de minutos limitado por n_minutos
            self.escritorios_ON[escritorio]['estado']                = 'pausa'#estado
            self.escritorios_ON[escritorio]['minutos_pausa']         = minutos_pausa#tiempo 
    def iniciar_tiempo_disponible(self,escritorio):
        self.escritorios_ON[escritorio]['contador_tiempo_disponible'] = iter(count(start=0, step=1))
        self.escritorios_ON[escritorio]['estado']                     = 'disponible'#     
    def iterar_escritorios_bloqueados(self, escritorios_bloqueados: List[str], tipo_inactividad:str = "Porcentaje"):

        for escri_bloq in escritorios_bloqueados:
            #ver si está en atención:
            if self.escritorios_ON[escri_bloq]['estado'] == 'atención':                
                #avanzamos en un minuto el tiempo de atención
                tiempo_atencion = next(self.escritorios_ON[escri_bloq]['contador_tiempo_atencion'], None)
                #si terminó la atención
                if tiempo_atencion is None: 
                    #iniciar pausa 
                    self.iniciar_pausa(escri_bloq, tipo_inactividad)
            #si el escritorio está en pausa:            
            elif self.escritorios_ON[escri_bloq]['estado'] == 'pausa':
                  #chequeamos si la inactividad es por pocentaje o pausas históricas
                 if tipo_inactividad == "Porcentaje":
                   #Avanzamos el contador de inactividad en un minuto
                   tiempo_inactividad = next(self.escritorios_ON[escri_bloq]['contador_inactividad'],None)
                   #si termina el tiempo de inactividad
                   if tiempo_inactividad is None:
                     #pasa a estado disponible
                     self.iniciar_tiempo_disponible(escri_bloq)
                 else: #pausas históricas                 
                    #iteramos contador_tiempo_pausa:
                    tiempo_pausa = next(self.escritorios_ON[escri_bloq]['contador_tiempo_pausa'], None)
                    if tiempo_pausa is None: 
                        #si termina tiempo en pausa pasa a estado disponible
                        self.iniciar_tiempo_disponible(escri_bloq)
    def iterar_escritorios_disponibles(self, escritorios_disponibles: List[str]):
        
        for escri_dispon in escritorios_disponibles:               
            #avanzamos en un minuto el tiempo que lleva disponible.
            tiempo_disponible = next(self.escritorios_ON[escri_dispon]['contador_tiempo_disponible'], None)
            if tiempo_disponible is not None:
            #guardar el tiempo que lleva disponible
                self.escritorios_ON[escri_dispon]['tiempo_actual_disponible'] = tiempo_disponible


def optuna_simular_v02(
    agenda_INPUT, 
    niveles_servicio_x_serie, 
    un_dia, 
    prioridades, 
    tipo_inactividad = "Porcentaje"):
  
  planificacion = copy.deepcopy(agenda_INPUT)  
  supervisor    = MisEscritorios_v02(inicio_tramo            = un_dia['FH_Emi'].min(),
                                    fin_tramo                = un_dia['FH_Emi'].max(),
                                    planificacion            = planificacion,
                                    niveles_servicio_x_serie = niveles_servicio_x_serie)
  tiempo_inicial         = list(generador_emisiones(un_dia))[0].FH_Emi#[0]['FH_Emi']
  generador_emisiones_in = generador_emisiones(un_dia)
  contador_tiempo        = timestamp_iterator(tiempo_inicial)
  reloj_simulacion       = next(contador_tiempo)
  fila                   = pd.DataFrame()
  registros_atenciones   = pd.DataFrame()
  SLA_df                 = pd.DataFrame()
  SLA_index              = 0
  Espera_index           = 0 
  Espera_df              = pd.DataFrame() 
  una_emision            = next(generador_emisiones_in)
  emi                    = una_emision['FH_Emi']
  num_emisiones          = un_dia.shape[0]
  
  for _ in range(2*num_emisiones):    
    supervisor.aplicar_agenda(hora_actual=  reloj_simulacion, agenda = planificacion)
    if not mismo_minuto(emi, reloj_simulacion):
      
      reloj_simulacion  = next(contador_tiempo)
      #flag seteada avanzar
      #print(f"avanza el reloj un minuto, nuevo tiempo: {reloj_simulacion}, avanza tiempo de espera y tiempo en escritorios bloqueados (atencion y pausa)")
      #print("tiempos de espera incrementados en un minuto")
      fila['espera'] += 1
      if (supervisor.filtrar_x_estado('atención') or  supervisor.filtrar_x_estado('pausa')):
          en_atencion            = supervisor.filtrar_x_estado('atención') or []
          en_pausa               = supervisor.filtrar_x_estado('pausa') or []
          escritorios_bloqueados = set(en_atencion + en_pausa)            
          #print(f"escritorios ocupados (bloqueados) por servicio: {escritorios_bloqueados}")
          #Avanzar un minuto en todos los tiempos de atención en todos los escritorios bloquedos  
          escritorios_bloqueados_conectados    = [k for k,v in supervisor.escritorios_ON.items() if k in escritorios_bloqueados]
                  
          supervisor.iterar_escritorios_bloqueados(escritorios_bloqueados_conectados, tipo_inactividad = tipo_inactividad )

      if disponibles:= supervisor.filtrar_x_estado('disponible'):
          conectados_disponibles       = [k for k,v in supervisor.escritorios_ON.items() if k in disponibles]
          supervisor.iterar_escritorios_disponibles(conectados_disponibles)
    else:
          #print(
          #f"emisioń dentro del mismo minuto (diferencia {abs(emi-reloj_simulacion).total_seconds()} seg.), actualizamos fila, gestionamos escritorios y pasamos a la siguiente emisión")    
          #poner la emisión en una fila de df
          emision_cliente = pd.DataFrame(una_emision).T
          #insertar una nueva columna de tiempo de espera y asignar con cero.
          emision_cliente['espera'] = 0
          #concatenar la emisión a la fila de espera
          fila = pd.concat([fila, emision_cliente]).reset_index(drop=True)
          #fila_para_SLA = copy.deepcopy(fila[['FH_Emi', 'IdSerie', 'espera']])
          #print(f"fila actualizada: \n{fila[['FH_Emi','IdSerie','espera']]}")

          if not fila.empty:
              if disponibles:= supervisor.filtrar_x_estado('disponible'):
                  #extraer las skills de los escritorios conectados que están disponibles
                  #conectados_disponibles       = [k for k,v in supervisor.escritorios_ON.items() if k in disponibles]
                  conectados_disponibles       = balancear_carga_escritorios(
                                                                              {k: {'numero_de_atenciones':v['numero_de_atenciones'],
                                                                                  'tiempo_actual_disponible': v['tiempo_actual_disponible']} 
                                                                              for k,v in supervisor.escritorios_ON.items() if k in disponibles}
                                                                              )    

                  skills_disponibles           = {k:v['skills'] for k,v in supervisor.escritorios_ON.items() if k in disponibles}
                  configuraciones_disponibles  = {k:v['configuracion_atencion'] for k,v in supervisor.escritorios_ON.items() if k in disponibles}

                  for un_escritorio in conectados_disponibles:

                      configuracion_atencion = supervisor.escritorios_ON[un_escritorio]['configuracion_atencion']
                      #print(f"buscando cliente para {un_escritorio} con {configuracion_atencion}")
                      fila_filtrada          = fila[fila['IdSerie'].isin(supervisor.escritorios_ON[un_escritorio].get('skills', []))]#filtrar_fila_por_skills(fila, supervisor.escritorios_ON[un_escritorio])
                      #print(f"en base a las skills: {supervisor.escritorios_ON[un_escritorio].get('skills', [])}, fila_filtrada \n{fila_filtrada}")
                      if  fila_filtrada.empty:
                              #print("No hay match entre idSeries en fila y skills del escritorio, saltar al siguiente escritorio")
                              continue #
                      elif configuracion_atencion == "Alternancia":
                          #print("----Alternancia------")
                          #print(
                          #    f"prioridades: {prioridades} skills: {supervisor.escritorios_ON[un_escritorio]['skills']} \n{supervisor.escritorios_ON[un_escritorio]['pasos_alternancia'].pasos}"                       
                          #)                        
                          cliente_seleccionado = supervisor.escritorios_ON[un_escritorio]['pasos_alternancia'].buscar_cliente(fila_filtrada)
                          #break
                      elif configuracion_atencion == "FIFO":
                          cliente_seleccionado = FIFO(fila_filtrada)
                          #print(f"cliente_seleccionado por {un_escritorio} en configuración FIFO: su emisión fue a las: {cliente_seleccionado.FH_Emi}")
                          #break
                      elif configuracion_atencion == "Rebalse":
                          cliente_seleccionado = extract_highest_priority_and_earliest_time_row(fila_filtrada, prioridades)
                          #print(f"cliente_seleccionado por {un_escritorio} en configuración Rebalse: su emisión fue a las: {cliente_seleccionado.FH_Emi}")
                      fila = remove_selected_row(fila, cliente_seleccionado)
                      supervisor.iniciar_atencion(un_escritorio, cliente_seleccionado)
                      un_cliente   = pd.DataFrame(cliente_seleccionado[['FH_Emi', 'IdSerie', 'espera','IdEsc','T_Ate']]).T
                      
                      
                      registros_atenciones   =  pd.concat([registros_atenciones, un_cliente])#.reset_index(drop=True)

          try:
              #Iterar a la siguiente emisión
              #supervisor.aplicar_agenda(hora_actual=  reloj_simulacion, agenda = agenda)    
              una_emision            = next(generador_emisiones_in)
              emi                    = una_emision['FH_Emi']
              #print(f"siguiente emisión {emi}")
          except StopIteration:
              #print(f"-----------------------------Se acabaron las emisiones en la emision numero {numero_emision} ---------------------------")
              break   
  return pd.concat([fila[['FH_Emi','IdSerie','espera']], registros_atenciones]).sort_values(by='FH_Emi', inplace=False).reset_index(drop=True), len(fila)


# Objetivos optuna

In [5]:

import optuna
import numpy as np
dataset = DatasetTTP.desde_csv_atenciones("data/fonasa_monjitas.csv.gz")
un_dia = dataset.un_dia("2023-05-15").sort_values(by='FH_Emi', inplace=False)
skills   = obtener_skills(un_dia)
series   = sorted(list({val for sublist in skills.values() for val in sublist}))
modos    = ['Rebalse','Alternancia', 'Rebalse']
atributos_series = atributos_x_serie(ids_series=series, 
                                    sla_porcen_user=None, 
                                    sla_corte_user=None, 
                                    pasos_user=None, 
                                    prioridades_user=None)

niveles_servicio_x_serie = {atr_dict['serie']:
                           (atr_dict['sla_porcen']/100, atr_dict['sla_corte']/60) 
                           for atr_dict in atributos_series}

prioridades =       {atr_dict['serie']:
                    atr_dict['prioridad']
                    for atr_dict in atributos_series}

def objective(trial, 
    optimizar: str, 
    un_dia : pd.DataFrame,  # IdOficina  IdSerie  IdEsc, FH_Emi, FH_Llama  -- Deberia llamarse 'un_tramo'
    subsets, # [(5,), (10,), (11,), (12,), (14,), (17,), (5, 10), (5, 11), (5, 12), (5, 14), (5, 17), (10, 11),  <...> 14, 17), (5, 10, 12, 14, 17), (5, 11, 12, 14, 17), (10, 11, 12, 14, 17), (5, 10, 11, 12, 14, 17)]
    niveles_servicio_x_serie,  # {5: (0.34, 35), 10: (0.34, 35), 11: (0.7, 45), 12: (0.34, 35), 14: (0.34, 35), 17: (0.6, 30)}
    prioridades:dict,
    modos_atenciones : list = ["Alternancia", "FIFO", "Rebalse"],
    minimo_escritorios: int = 2,
    maximo_escritorios: int = 5,
    ):    
    try:

        bool_vector              = [trial.suggest_categorical(f'escritorio_{i}', [True, False]) for i in range(maximo_escritorios)]
        #Restricción de minimo de escritorios
        assert sum(bool_vector) >= minimo_escritorios, f"No cumple con minimo_escritorios: {minimo_escritorios}."
        
        str_dict                 = {i: trial.suggest_categorical(f'{i}',         modos_atenciones) for i in range(maximo_escritorios)} 
        subset_idx               = {i: trial.suggest_int(f'ids_{i}', 0, len(subsets) - 1) for i in range(maximo_escritorios)}   
        #prioridades              =  prioridad_x_serie(niveles_servicio_x_serie, 2, 1) 
        planificacion            =  {} # Arma una planificacion con espacios parametricos. 
        inicio                   =  str(un_dia.FH_Emi.min().time())#'08:33:00'
        termino                  =  str(un_dia.FH_Emi.max().time())#'14:33:00'

        for key in str_dict.keys():
            if bool_vector[key]:
                inner_dict = {
                    'inicio': inicio,
                    'termino': termino,
                    'propiedades': {
                        'skills':list(subsets[subset_idx[key]]), # Set -> Lista, para el subset 'subset_idx', para el escritorio 'key'
                        #'skills': list(subset_dict[key]),
                        'configuracion_atencion': str_dict[key], # FI FAI FO FU
                        #'porcentaje_actividad':  np.random.randint(75, 90)/100,
                    }
                }
                planificacion[str(key)] = [inner_dict] # NOTE: Es una lista why -- Config por trial por tramo del escritorio 

        trial.set_user_attr('planificacion', planificacion) # This' actually cool 
        registros_atenciones, l_fila    =  optuna_simular_v02(planificacion, niveles_servicio_x_serie, un_dia, prioridades, tipo_inactividad = "historica") 
        registros_atenciones['IdSerie'] = registros_atenciones['IdSerie'].astype(int) 
        registros_x_serie               = [registros_atenciones[registros_atenciones.IdSerie==s] for s in series]
        
        
        pocentajes_SLA        = [int(100*v[0])for k,v in niveles_servicio_x_serie.items()]
        mins_de_corte_SLA     = [int(v[1])for k,v in niveles_servicio_x_serie.items()]        
        df_pairs              = [(sla_x_serie(r_x_s, '1H', corte = corte, factor_conversion_T_esp=1), s) 
                                    for r_x_s, s, corte in zip(registros_x_serie, series, mins_de_corte_SLA)]
        porcentajes_reales    = {f"serie: {serie}": np.mean(esperas.espera) for ((demandas, esperas), serie) in df_pairs} 
        dif_cuadratica        = {k:(v-p)**2 for ((k,v),p) in zip(porcentajes_reales.items(),pocentajes_SLA)}
        #Objetivos:    
        #La mayor prioridad es el entero más chico    
        maximizar_SLAs        = tuple(np.array(tuple(prioridades.values()))*np.array(tuple(dif_cuadratica.values())))#Ponderado por prioridad
        minimizar_escritorios = (sum(bool_vector),)
        minimizar_skills      = (extract_skills_length(planificacion),)
        
        if optimizar == "SLA":
            
            print(f"maximizar_SLAs {maximizar_SLAs}")
            return  maximizar_SLAs
        
        elif optimizar == "SLA + escritorios":
            
            print(f"maximizar_SLAs y minimizar_escritorios {maximizar_SLAs, minimizar_escritorios}")
            return  maximizar_SLAs + minimizar_escritorios
        
        elif optimizar == "SLA + skills":
            
            print(f"maximizar_SLAs y minimizar_skills {maximizar_SLAs, minimizar_skills}")
            return  maximizar_SLAs + minimizar_skills
        
        elif optimizar == "SLA + escritorios + skills":
            
            print(f"SLA + escritorios + skills {maximizar_SLAs, minimizar_escritorios, minimizar_skills}")
            return  maximizar_SLAs + minimizar_escritorios + minimizar_skills           
        
    except Exception as e:
        print(f"An exception occurred: {e}")
        raise optuna.TrialPruned()
    
#Si hay porcentaje_actividad no hay pausas  
#n es el numero de intervalos (equidistantes) de tiempo   
intervals  = get_time_intervals(un_dia, n = 4, porcentaje_actividad = .9) # Una funcion que recibe un dia, un intervalo, y un porcentaje de actividad para todos los intervalos
partitions = partition_dataframe_by_time_intervals(un_dia, intervals) # TODO: implementar como un static del simulador? 
optimizar  = "SLA + escritorios + skills" #"SLA" | "SLA + escritorios" | "SLA + skills" | "SLA + escritorios + skills"
n_objs = int(
            len(series)
            if optimizar == "SLA"
            else len(series) + 1
            if optimizar in {"SLA + escritorios", "SLA + skills"}
            else len(series) + 2
            if optimizar == "SLA + escritorios + skills"
            else None
        )
n_trials   = 1
#%%
storage = optuna.storages.get_storage("sqlite:///alejandro_objs_v3.db")
subsets = non_empty_subsets(sorted(list({val for sublist in skills.values() for val in sublist})))

for idx, part in enumerate(partitions):
    study_name = f"tramo_{idx}"
    study = optuna.multi_objective.create_study(directions= n_objs*['minimize'],
                                                study_name=study_name,
                                                storage=storage, load_if_exists=True)
    # TODO: sacar fuera
    # Optimize with a timeout (in seconds)
    study.optimize(lambda trial: objective(trial,
                                           optimizar                = optimizar,
                                           un_dia                   = part,
                                           subsets                  = subsets,
                                           niveles_servicio_x_serie = niveles_servicio_x_serie,
                                           prioridades              = prioridades,
                                           minimo_escritorios       = 2,
                                           maximo_escritorios       = 5
                                           ),
                   n_trials  = n_trials, #int(1e4),  # Make sure this is an integer
                   timeout   = 2*3600,   #  hours
                   )  # 


[I 2023-10-30 14:15:00,527] Using an existing study with name 'tramo_0' instead of creating a new one.
[I 2023-10-30 14:15:01,056] Trial 23 finished with values: (7235.900826446283,) with parameters: {'escritorio_0': False, 'escritorio_1': False, 'escritorio_2': True, 'escritorio_3': True, 'escritorio_4': False, '0': 'Alternancia', '1': 'Alternancia', '2': 'Rebalse', '3': 'FIFO', '4': 'Rebalse', 'ids_0': 7, 'ids_1': 46, 'ids_2': 21, 'ids_3': 34, 'ids_4': 19}.
[I 2023-10-30 14:15:01,104] Using an existing study with name 'tramo_1' instead of creating a new one.


SLA + escritorios + skills ((7235.900826446283, 952.3614103484834, 417.9753086419755, 280.05555555555566, 455.9480968858134, 40.33333333333313), (2,), (6,))


[I 2023-10-30 14:15:02,455] Trial 23 finished with values: (15.041666666666577,) with parameters: {'escritorio_0': False, 'escritorio_1': True, 'escritorio_2': True, 'escritorio_3': True, 'escritorio_4': False, '0': 'Alternancia', '1': 'Alternancia', '2': 'Rebalse', '3': 'Rebalse', '4': 'Rebalse', 'ids_0': 45, 'ids_1': 31, 'ids_2': 0, 'ids_3': 34, 'ids_4': 22}.
[I 2023-10-30 14:15:02,500] Using an existing study with name 'tramo_2' instead of creating a new one.


SLA + escritorios + skills ((15.041666666666577, 1736.5702479338836, 4356.0, 5089.283950617284, 1812.3265306122448, 4988.481481481482), (3,), (7,))


[I 2023-10-30 14:15:03,678] Trial 23 finished with values: (13659.918367346942,) with parameters: {'escritorio_0': True, 'escritorio_1': False, 'escritorio_2': True, 'escritorio_3': True, 'escritorio_4': False, '0': 'Alternancia', '1': 'Rebalse', '2': 'Rebalse', '3': 'FIFO', '4': 'Alternancia', 'ids_0': 62, 'ids_1': 18, 'ids_2': 16, 'ids_3': 47, 'ids_4': 37}.
[I 2023-10-30 14:15:03,729] Using an existing study with name 'tramo_3' instead of creating a new one.


SLA + escritorios + skills ((13659.918367346942, 6125.0, 676.0, 190.125, 771.0682629699643, 280.3333333333332), (3,), (12,))


[I 2023-10-30 14:15:04,881] Trial 23 finished with values: (23064.0,) with parameters: {'escritorio_0': True, 'escritorio_1': False, 'escritorio_2': True, 'escritorio_3': False, 'escritorio_4': True, '0': 'FIFO', '1': 'Alternancia', '2': 'Alternancia', '3': 'Alternancia', '4': 'Rebalse', 'ids_0': 36, 'ids_1': 21, 'ids_2': 61, 'ids_3': 0, 'ids_4': 47}.


SLA + escritorios + skills ((23064.0, 2292.8272625242316, 4578.777777777778, 8712.0, 10706.803324099723, 4332.0), (3,), (12,))


## Extracción de la planificación óptima

In [6]:
recomendaciones_db   = optuna.storages.get_storage("sqlite:///alejandro_objs_v3.db") # Objetivos de 6-salidas
resumenes            = optuna.study.get_all_study_summaries(recomendaciones_db)
nombres              = [s.study_name for s in resumenes if "tramo_" in s.study_name]

scores_studios = {}
for un_nombre in nombres:
    un_estudio            = optuna.multi_objective.load_study(study_name=un_nombre, storage=recomendaciones_db)
    trials_de_un_estudio  = un_estudio.get_trials(deepcopy=False) #or pareto trials??
    scores_studios        = scores_studios | {f"{un_nombre}":
        { trial.number: np.mean([x for x in trial.values if x is not None]) 
                for
                    trial in trials_de_un_estudio if trial.state == optuna.trial.TrialState.COMPLETE}
                    } 
trials_optimos          = extract_min_value_keys(scores_studios) # Para cada tramo, extrae el maximo, 
planificaciones_optimas = {}   
for k,v in trials_optimos.items():
    un_estudio               = optuna.multi_objective.load_study(study_name=k, storage=recomendaciones_db)
    trials_de_un_estudio     = un_estudio.get_trials(deepcopy=False)
    planificaciones_optimas  = planificaciones_optimas | {f"{k}":
        trial.user_attrs.get('planificacion')#calcular_optimo(trial.values)
                for
                    trial in trials_de_un_estudio if trial.number == v[0]
                    }   

planificacion_optima                =  plan_unico([plan for tramo,plan in planificaciones_optimas.items()])


### simulación con planificación óptima

In [7]:
registros_atenciones_optima, l_fila_optima =  optuna_simular_v02(planificacion_optima, niveles_servicio_x_serie, un_dia, prioridades, tipo_inactividad = "historica") 

In [8]:
registros_atenciones_optima, l_fila_optima

(                  FH_Emi IdSerie espera IdEsc T_Ate
 0    2023-05-15 08:40:11      12      9     7    96
 1    2023-05-15 08:40:54      14      9    10   157
 2    2023-05-15 08:41:07      14      9    12   458
 3    2023-05-15 08:41:18      14      8    11   861
 4    2023-05-15 08:41:27      14      8    10   381
 ..                   ...     ...    ...   ...   ...
 654  2023-05-15 14:12:59      10     18   NaN   NaN
 655  2023-05-15 14:13:23      10     17   NaN   NaN
 656  2023-05-15 14:14:26      12     16   NaN   NaN
 657  2023-05-15 14:18:32      10     12   NaN   NaN
 658  2023-05-15 14:30:23      17      0   NaN   NaN
 
 [659 rows x 5 columns],
 298)

In [ ]:
registros_atenciones_optima["IdEsc"].info()